# Импорты (нужно установить эти библиотеки)

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import csv

# Тестовый фрейм (одна строка, которая передается как post request)

In [2]:
test_request = {
    'BirthDate':            '1980-03-20',
    'education':            'Высшее - специалист',
    'employment_status':    'Собственное дело',
    'Value':                '10 и более лет',
    'JobStartDate':         '2012-10-01',
    'Position':             'Просто умный человек',
    #'MonthProfit':          150000.0,
    'MonthExpense':         '22500.0', 
    'Gender':               'sdfas', #'0',
    'Family_status':        'Никогда в браке не состоял(а)',
    'ChildCount':           '0',
    'MonthProfit':          np.nan,
    'SNILS':                '0',
    'Merch_code':           '15',
    'Loan_amount':          '48000.0',
    'Loan_term':            '6',
    'Goods_category':       'Medical_services'
}

# Образец Get и Post запроса

In [3]:
r = requests.get('http://127.0.0.1:5000/')
print(r.status_code, r.reason)

200 OK


In [4]:
json_data = json.dumps(test_request).encode('utf8')
r = requests.post(
    #'http://158.160.135.101:5000/AskOraqul/', data=json_data)
    'http://127.0.0.1:5000/AskOraqul/', data=json_data)
print(r.status_code, r.reason)
resp = r.content.decode()[1:-1].split(',')
BankA_decision = round(float(resp[0]), 2)
BankB_decision = round(float(resp[1]), 2)
BankC_decision = round(float(resp[2]), 2)
BankD_decision = round(float(resp[3]), 2)
BankE_decision = round(float(resp[4]), 2)
print(BankA_decision, BankB_decision, BankC_decision, BankD_decision, BankE_decision)

200 OK
0.59 0.57 0.54 0.55 0.47


# Post запросы в цикле с перебором строк из загружаемого датафрейма

In [23]:
with open('../data/model_val.csv') as f:
    next(f)  # Skip the header
    reader = csv.reader(f, skipinitialspace=True, delimiter=';')
    for row in reader:
        test_request = {} 
        test_request['BirthDate']           = row[1]
        test_request['education']           = row[2]
        test_request['employment_status']   = row[3]
        test_request['Value']               = row[4]
        test_request['JobStartDate']        = row[5]
        test_request['Position']            = row[6]
        test_request['MonthProfit']         = row[7]
        test_request['MonthExpense']        = row[8]
        test_request['Gender']              = row[9]
        test_request['Family_status']       = row[10]
        test_request['ChildCount']          = row[11]
        test_request['SNILS']               = row[12]
        test_request['Merch_code']          = row[18]
        test_request['Loan_amount']         = row[19]
        test_request['Loan_term']           = row[20]
        test_request['Goods_category']      = row[21]
        json_data = json.dumps(test_request).encode('utf8')
        r = requests.post(
            'http://158.160.135.101:5000/AskOraqul/', data=json_data)
            #'http://127.0.0.1:5000/AskOraqul/', data=json_data)
        print(r.status_code, r.reason)
        resp = r.content.decode()[1:-1].split(',')
        BankA_decision = round(float(resp[0]), 2)
        BankB_decision = round(float(resp[1]), 2)
        BankC_decision = round(float(resp[2]), 2)
        BankD_decision = round(float(resp[3]), 2)
        BankE_decision = round(float(resp[4]), 2)
        print(BankA_decision, BankB_decision, BankC_decision, BankD_decision, BankE_decision)
        

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


KeyboardInterrupt: 

In [7]:
# Загружаем тестовую выборку
df = pd.read_csv(
    filepath_or_buffer='../data/model_val.csv',
    sep=';',
    index_col='SkillFactory_Id')
banks = ['BankA_decision',
             'BankB_decision',
             'BankC_decision',
             'BankD_decision',
             'BankE_decision']
df = df.drop(banks, axis='columns')
df = df.dropna(how='all')
df.rename(columns={
    "employment status": "employment_status",
    "Family status": "Family_status"}, 
    inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1251 entries, 2443.0 to 768.0
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   education          1251 non-null   object 
 1   employment_status  1251 non-null   object 
 2   Position           1251 non-null   object 
 3   Gender             1251 non-null   float64
 4   Family_status      1251 non-null   object 
 5   ChildCount         1251 non-null   float64
 6   SNILS              1251 non-null   float64
 7   Merch_code         1251 non-null   float64
 8   Goods_category     1251 non-null   object 
 9   Age                1251 non-null   int64  
 10  NumValue           1251 non-null   int64  
 11  Payment_to_income  1251 non-null   float64
dtypes: float64(5), int64(2), object(5)
memory usage: 127.1+ KB


In [9]:
# Отправляем произвольные 50 строк из загруженного датафрейма
frame_for_send = df.sample(50).to_dict('records')
for i in range(len(frame_for_send)):
    #json_data = json.dumps(frame_for_send[i]).encode('utf8')
    json_data = df.iloc[i].to_json().encode('utf8')
    r = requests.post(
        #'http://158.160.135.101:5000/AskOraqul/', data=json_data)
        'http://127.0.0.1:5000/AskOraqul/', data=json_data)
    print(r.status_code, r.reason)
    resp = r.content.decode()[1:-1].split(',')
    BankA_decision = round(float(resp[0]), 2)
    BankB_decision = round(float(resp[1]), 2)
    BankC_decision = round(float(resp[2]), 2)
    BankD_decision = round(float(resp[3]), 2)
    BankE_decision = round(float(resp[4]), 2)
    print(BankA_decision, BankB_decision, BankC_decision, BankD_decision, BankE_decision)


200 OK
0.86 0.68 0.31 0.75 0.9
200 OK
0.86 0.63 0.49 0.41 0.76
200 OK
0.85 0.66 0.47 0.4 0.7
200 OK
0.85 0.66 0.74 0.6 0.75
200 OK
0.87 0.75 0.35 0.87 0.79
200 OK
0.34 0.85 0.64 0.79 0.45
200 OK
0.86 0.73 0.56 0.58 0.58
200 OK
0.91 0.75 0.5 0.7 0.84
200 OK
0.9 0.72 0.76 0.71 0.73
200 OK
0.56 0.76 0.81 0.78 0.37
200 OK
0.81 0.5 0.61 0.47 0.96
200 OK
0.89 0.29 0.68 0.6 0.44
200 OK
0.73 0.72 0.42 0.63 0.6
200 OK
0.88 0.59 0.63 0.43 0.87
200 OK
0.92 0.75 0.46 0.68 0.75
200 OK
0.78 0.61 0.58 0.41 0.82
200 OK
0.85 0.75 0.56 0.43 0.71
200 OK
0.64 0.55 0.82 0.91 0.99
200 OK
0.79 0.37 0.51 0.6 0.96
200 OK
0.85 0.65 0.49 0.41 0.76
200 OK
0.92 0.42 0.68 0.61 0.4
200 OK
0.87 0.38 0.45 0.75 0.93
200 OK
0.84 0.68 0.77 0.48 0.73
200 OK
0.54 0.66 0.41 0.66 0.57
200 OK
0.53 0.31 0.29 0.73 0.58
200 OK
0.88 0.55 0.57 0.44 0.86
200 OK
0.72 0.82 0.9 0.58 0.68
200 OK
0.9 0.55 0.56 0.65 0.34
200 OK
0.87 0.71 0.69 0.43 0.64
200 OK
0.95 0.51 0.93 0.94 0.3
200 OK
0.87 0.74 0.7 0.75 0.59
200 OK
0.77 0.66 0.4 0.6